# Integrated Gradients
入力特徴量の各寄与度を積分により計算する手法  
baselineと実際の入力の間を積分して、各特徴量の寄与度を求める  
各入力要素が予測にどの程度影響しているかのスコアが得られ、数値的に解釈できる

In [2]:
import torch
import torch.nn as nn
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成（回帰タスク： y = 3*x + 2 にノイズ）
X = np.linspace(0, 10, 100).reshape(-1, 1)
y = 3 * X + 2 + np.random.normal(0, 1, X.shape)

# NumPy → Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size=16):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = SimpleMLP(hidden_size=16)

# 簡単な訓練（ここでは数エポックのみ）
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()
for epoch in range(200):
    model.train()
    optimizer.zero_grad()
    preds = model(X_tensor)
    loss = criterion(preds, y_tensor)
    loss.backward()
    optimizer.step()

print("Training completed. Final loss:", loss.item())


Training completed. Final loss: 0.8066921234130859


In [4]:
from captum.attr import IntegratedGradients

# モデルを評価モードに
model.eval()

# Integrated Gradientsのインスタンス作成
ig = IntegratedGradients(model)

# 対象となるサンプル（例として、サンプルインデックス10）
input_tensor = X_tensor[10:11]  # バッチサイズ1にする
baseline = torch.zeros_like(input_tensor)  # baselineとしてゼロベクトルを使用

# 属性（各特徴量の寄与度）の計算
attributions, delta = ig.attribute(input_tensor, baseline, target=0, return_convergence_delta=True)

print("Attributions:", attributions.numpy())
print("Convergence Delta:", delta.numpy())


Attributions: [[2.74365663]]
Convergence Delta: [-0.00064946]
